# Imports

In [7]:
import os
import json 

# Settings

In [8]:

setting_file = os.path.abspath(os.path.join('..', 'settings.json'))

with open(setting_file, 'r') as f:
    settings = json.load(f)


# Connect to Database
Use SqlAlchemy to autoload tables

In [9]:
# use sqlalchemy and reflection to get the table names
from sqlalchemy import create_engine, MetaData, Table, func
from sqlalchemy.orm import sessionmaker

# create the engine
engine = create_engine(settings['sqlalchemy_database_uri'])

# Reflect the tables
metadata = MetaData()
metadata.reflect(bind=engine)

# create connection
conn = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()

# load tables
Prototypes = Table('prototypes', metadata, autoload_with=engine)
Analysis = Table('analysis', metadata, autoload_with=engine)
Sample = Table('sample', metadata, autoload_with=engine)
Tag = Table('tag', metadata, autoload_with=engine)
SampleTag = Table('sample_tag', metadata, autoload_with=engine)

# Retrieve and write prototype code to file

In [10]:
# get one prototype
prototype = session.query(Prototypes).first()

tmp_dir = os.path.abspath('tmp')
os.makedirs(tmp_dir, exist_ok=True)

# write the prototype to a file
prototype_source = os.path.join(tmp_dir, 'prototype.c')
with open(prototype_source, 'w') as f:
    f.write(prototype.code)


# Create XML file of prototype code

In [11]:
srcml_client = settings['srcml_client']

# use subprocess to run the srcml client
import subprocess


# compile the code
prototype_xml = os.path.join(tmp_dir, 'prototype.xml')            
result = subprocess.run([srcml_client, prototype_source, '-o', prototype_xml], capture_output=True, text=True)

# check if the command was successful
if result.returncode != 0:
    print(result.stderr)
    raise Exception('Failed to run srcml client')
else:
    print(result.stdout)
